In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df0 = pd.read_csv('건물_30년 초과.csv')
df0

,자치구,계약일,건물면적(㎡),층,KOSPI종가,Fed기준금리,국제유가(Dubai)($/배럴),부동산거래량,국고채5년(평균),국고채10년(평균),...,BOK기준금리,건물연식,GDP대비가계부채비율,환율종가,선행지수순환변동치,EPU,장단기금리spread,금값종가,LTV,물건금액(만원)
0,강남구,2016-01-01,50.67,4,1961.310059,0.34,26.9,9894,1.77,2.03,...,1.50,34,83.3,1175.45,99.6,177.148845,0.14,1060.74,0.7,87000
1,영등포구,2016-01-02,55.90,3,1961.310059,0.34,26.9,9894,1.77,2.03,...,1.50,33,83.3,1175.45,99.6,177.148845,0.14,1060.74,0.7,28500
2,서대문구,2016-01-03,66.12,6,1961.310059,0.34,26.9,9894,1.77,2.03,...,1.50,47,83.3,1175.45,99.6,177.148845,0.14,1060.74,0.7,15200
3,강남구,2016-01-04,25.27,3,1918.760010,0.34,26.9,9894,1.77,2.03,...,1.50,34,83.3,1190.35,99.6,177.148845,0.14,1074.70,0.7,51000
4,강동구,2016-01-04,41.62,2,1918.760010,0.34,26.9,9894,1.77,2.03,...,1.50,33,83.3,1190.35,99.6,177.148845,0.14,1074.70,0.7,41000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65821,서대문구,2022-12-30,54.02,2,2236.399902,4.10,77.2,3698,3.59,3.54,...,3.25,48,105.0,1260.92,99.1,169.662386,0.39,1824.40,0.4,53459
65822,노원구,2022-12-31,41.30,12,2236.399902,4.10,77.2,3698,3.59,3.54,...,3.25,34,105.0,1260.92,99.1,169.662386,0.39,1824.40,0.4,39000
65823,송파구,2022-12-31,100.82,1,2236.399902,4.10,77.2,3698,3.59,3.54,...,3.25,34,105.0,1260.92,99.1,169.662386,0.39,1824.40,0.0,185000
65824,강남구,2022-12-31,76.79,2,2236.399902,4.10,77.2,3698,3.59,3.54,...,3.25,43,105.0,1260.92,99.1,169.662386,0.39,1824.40,0.0,184000


In [3]:
df0 = df0[['자치구','계약일','건물면적(㎡)', '층', 'Fed기준금리',
        '건물연식', 'GDP대비가계부채비율',
        'EPU', '장단기금리spread','LTV', '물건금액(만원)']]

In [4]:
# Train : 2016-2020
# Test : 2021-2022

df0['계약일'] = pd.to_datetime(df0['계약일'])

C:\Users\user\AppData\Local\Temp\ipykernel_6232\3790176075.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df0['계약일'] = pd.to_datetime(df0['계약일'])


In [5]:
train_df = df0.loc[df0['계약일'] < '2021']

In [6]:
test_df = df0.loc[df0['계약일'] >= '2021']

In [7]:
# Train Split
# feature & target  split

X_train_df = train_df.drop('물건금액(만원)', axis=1)
y = train_df['물건금액(만원)']

# X_train : 숫자형 컬럼만 
X_train = X_train_df.select_dtypes([np.number])

In [8]:
# Test Split
# feature & target  split

X_test_df = test_df.drop('물건금액(만원)', axis=1)
y_test = test_df['물건금액(만원)']

# X_test : 숫자형 컬럼만 
X_test = X_test_df.select_dtypes([np.number])

In [9]:
# minMax
from sklearn.preprocessing import MinMaxScaler

# Scaler 객체 생성
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# DataFrame
X_scaled = pd.DataFrame(X_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [10]:
# y -> log 변환

# Train
y_scaled = np.log10(y)
# y_scaled

# Test
y_test_scaled = np.log10(y_test)
y_test_scaled

53765    5.439333
53766    5.250420
53767    4.995635
53768    5.129690
53769    5.332438
           ...   
65821    4.728021
65822    4.591065
65823    5.267172
65824    5.264818
65825    5.139879
Name: 물건금액(만원), Length: 12061, dtype: float64

In [11]:
# 교차 검증 수행
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

mlr = LinearRegression()

cross_val_scores = cross_val_score(mlr, X_scaled, y_scaled, scoring='neg_mean_squared_error', cv=5)
cross_val_rmse_scores = np.sqrt(-cross_val_scores)
avg_cross_val_rmse = np.mean(cross_val_rmse_scores)
print('Average Cross-Validated RMSE: {:.3f}'.format(avg_cross_val_rmse))
print(f'교차검증별 MSE : {np.round(-cross_val_scores,4)}')
print(f'교차검증 평균 MSE : {np.round(np.mean(-cross_val_scores),5)}')

Average Cross-Validated RMSE: 0.238
교차검증별 MSE : [0.0626 0.0579 0.0695 0.0511 0.0449]
교차검증 평균 MSE : 0.05718


In [12]:
X_scaled.columns

Index(['건물면적(㎡)', '층', 'Fed기준금리', '건물연식', 'GDP대비가계부채비율', 'EPU', '장단기금리spread',
       'LTV'],
      dtype='object')

In [13]:
X_test_scaled=X_test_scaled[X_scaled.columns]

In [14]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate
mlr = LinearRegression()
# 모델을 훈련하고 교차 검증을 수행
cv_results = cross_validate(mlr, X_scaled, y_scaled, cv=5, return_estimator=True)

# 교차 검증 결과에서 각 폴드에서 훈련된 모델을 가져옴
trained_models = cv_results['estimator']
# 각 폴드에서 훈련된 모델을 테스트 데이터에 적용
for model in trained_models:
    y_pred = model.predict(X_test_scaled)
    # 예측 결과를 활용하여 원하는 작업을 수행
    mse = mean_squared_error(y_test_scaled, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    print('MSE : {0:}, RMSE : {1: .3f}, MAE:{1: .3f}'.format(mse, rmse, mae))
    print('Variance score : {0: .3f}'.format(r2_score(y_test_scaled, y_pred)))
    print('\n')

MSE : 0.05421744020403806, RMSE :  0.233, MAE: 0.233
Variance score :  0.456


MSE : 0.04473399905538469, RMSE :  0.212, MAE: 0.212
Variance score :  0.551


MSE : 0.03876990650611974, RMSE :  0.197, MAE: 0.197
Variance score :  0.611


MSE : 0.03997839647687814, RMSE :  0.200, MAE: 0.200
Variance score :  0.599


MSE : 0.0540141284906817, RMSE :  0.232, MAE: 0.232
Variance score :  0.458




In [15]:
from sklearn.model_selection import cross_validate

y_pred = trained_models[3].predict(X_test_scaled)

mse = mean_squared_error(y_test_scaled, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_scaled,y_pred)

print('MSE : {0:}, RMSE : {1: .3f}, MAE:{1: .3f}'.format(mse, rmse, mae))
print('Variance score : {0: .3f}'.format(r2_score(y_test_scaled, y_pred)))

MSE : 0.03997839647687814, RMSE :  0.200, MAE: 0.200
Variance score :  0.599


In [16]:
# test 계수 결과 DF 

test_result = pd.DataFrame(data= X_test_scaled.columns)
test_result.columns = ['features']

test_result['coefficient'] = trained_models[3].coef_

test_result.sort_values('coefficient', ascending=False)

,features,coefficient
0,건물면적(㎡),1.739757
3,건물연식,0.084460
6,장단기금리spread,0.073746
1,층,0.061983
5,EPU,-0.050322
4,GDP대비가계부채비율,-0.147844
2,Fed기준금리,-0.204212
7,LTV,-0.307297
